# Config

In [ ]:
!pip install -q langchain-qdrant qdrant-client langchain langchain-community langchain_google_vertexai
!pip install -q unstructured transformers sentence-transformers openai rank-bm25 langchain-experimental langchain_openai langchain_huggingface google-generativeai python-dotenv langchain_google_genai


In [ ]:
!pip install -q ranx

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


# New Section

In [ ]:

import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from pathlib import Path
from typing import Literal

from pydantic import BaseModel, Field
# from dotenv import load_dotenv
from google.colab import userdata

from google.oauth2 import service_account
from langchain_google_vertexai import ChatVertexAI
# load_dotenv(Path("./.env"))
# from google.colab import userdata


# # === 1. Setup Vertex AI with credentials ===
credentials_path = "/content/drive/MyDrive/env/tdtuchat-16614553b756.json"
credentials = service_account.Credentials.from_service_account_file(credentials_path)

##########
llm = ChatVertexAI(
    model="gemini-1.5-pro",
    streaming= True,
    temperature=0.2,
    credentials=credentials,
    max_retries=5
)



In [ ]:
import time
import pickle
import os
from ranx import Run, evaluate
import pandas as pd
from ranx import Qrels
from collections import defaultdict


In [ ]:
# from QueryRouter import QueryRouter
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings
import google.generativeai as genai
from langchain_qdrant import QdrantVectorStore
import os
from pathlib import Path
# from dotenv import load_dotenv
# load_dotenv(Path("./.env"))

# Query router

In [ ]:

class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )
class UniversityCode(BaseModel):
    """Route a query to the corresponding university code."""

    university_code: Literal[
        "NTTU", "HCMUE", "UMP", "HCMUTE", "VLU",
        "PNTU", "FTU2", "TDTU", "UEH", "FPT",
        "BKU", "HCMUS", "OU", "UIT", "UFM"
    ] = Field(
        ...,
        description="University code corresponding to the query, if none: TDTU"
    )
class QueryRouter:
     def __init__(self, llm= None):
          self.llm = llm or ChatVertexAI(
                      model="gemini-1.5-pro",
                      streaming= True,
                      temperature=0.2,
                      credentials=credentials,
                      max_retries=5
                  )

          self.route_prompt = PromptTemplate(
                    input_variables=["query"],
                    template= """Dựa vào truy vấn(câu hỏi) từ người dùng: {query}
                    Hãy phân thuộc dạng truy vấn vào: vectorstore hay web search.
                    Vectorstore bao gồm tất cả các thông tin liên quan đến tất cả các vấn đề tuyển sinh của các trường đại học tại Việt Nam: bao gồm điểm chuẩn qua các năm, học phí, thông tin trường đại học, thông tin ngành, ...
                    Nếu nội dung câu truy vấn của người dùng liên quan đến các vấn đề trên thì phần loại là vectorstore, còn lại là websearch """
          )
          self.knowledge_prompt = PromptTemplate(
               input_variables=["query"],
               template="""Phân loại truy vấn thuộc về trường nào trong 15 trường sau:
               1.Trường đại học Nguyễn Tất Thành -> NTTU
               2.Trường đại học Sư Phạm TP HCM -> HCMUE
               3.Trường đại học Y Dược TP HCM -> UMP
               4.Trường đại học Tài Chính - Marketing -> UFM
               5.Trường đại học Văn Lang -> VLU
               6.Trường đại học Y Khoa Phạm Ngọc Thạch -> PNTU
               7. Trường đại học Sư Phạm Kỹ Thuật TP HCM -> HCMUTE
               8. Trường đại học Ngoại Thương TP HCM -> FTU2
               9. Trường đại học Tôn Đức Thắng -> TDTU
               10. Trường đại học Kinh Tế TP HCM -> UEH
               11. Trường đại học FPT -> FPT
               12. Trường đại học Bách Khoa TP HCM -> BKU
               13. Trường đại học Khoa Học Tự Nhiên TP HCM -> HCMUS
               14. Trường đại học Mở TP HCM -> OU
               15. Trường đại học Công Nghệ Thông Tin TP HCM -> UIT
               Query: {query}
               Chỉ in ra tên trường đã được phân loại:
               ví dụ: VLU
               """
          )

     def classify(self, query):
          print("Phân loại truy vấn")
          structured_llm_router = self.llm.with_structured_output(RouteQuery)
          question_router = self.route_prompt | structured_llm_router
          response = question_router.invoke(query)
          return response

     def UniversityRouting(self, query):
          structured_llm_router = self.llm.with_structured_output(UniversityCode)
          question_router = self.knowledge_prompt | structured_llm_router
          response = question_router.invoke(query)
          return response


In [ ]:
# Test router
# classifier = QueryRouter()
# source = classifier.classify("Chỉ tiêu tuyển sinh đại học Nguyễn Tất Thành 2022?")
# print((source.datasource))
# print(classifier.UniversityRouting({"query": "Thông tin trường Ngoai Thuong"}).university_code)

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-document-embedding",
    model_kwargs={"trust_remote_code": True}
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
class BaseRetrievalStrategy:
     def __init__(self, llm = None):
          self.embeddings = embedding_model
          self.llm = llm or ChatVertexAI(
                                model="gemini-1.5-pro",
                                streaming= True,
                                temperature=0.2,
                                credentials=credentials,
                                max_retries=5
                            )
          self.classifier = QueryRouter()

     def retrieve(self, query, k=3):
          return self.db.similarity_search(query, k=k)

# Navie Retrieval

In [ ]:

# Version 1
class UniversityRetrievalStrategy(BaseRetrievalStrategy):

     def retrieve(self, query: str, k = 3)-> list:
          university = self.classifier.UniversityRouting(query).university_code
          print(f'Truy vấn thuộc về trường: {university}')

          client = QdrantClient(
               url=userdata.get('qdrant_url2'),
               api_key=userdata.get('qdrant_api2'),
               prefer_grpc=True
          )

          vector_store = QdrantVectorStore(
               client=client,
               collection_name=str(university),
               embedding=embedding_model
          )
          return vector_store.similarity_search(query, k)



## Test NV

In [ ]:

retriever = UniversityRetrievalStrategy()
query = "Điểm chuẩn ngành Ngôn ngữ Anh 2021"
docs = retriever.retrieve(query, k= 3)
print(docs)

Truy vấn thuộc về trường: TDTU
[Document(metadata={'source': 'Admission\\TDTU\\2021_tuyển sinh.md', '_id': '4f226c38-1b71-4c6f-9f5e-4666f30ec8df', '_collection_name': 'TDTU'}, page_content='Đối tượng 6: Thí sinh có chứng chỉ SAT, A-Level, IB, ACT ưu tiên xét tuyển vào chương trình đại học bằng tiếng Anh:\n\nThí sinh tốt nghiệp THPT từ năm 2021 trở về trước (có xác nhận đạt trình độ tương đương trình độ THPT của Việt Nam), có các chứng chỉ SAT, A-Level, IB, ACT còn giá trị sử dụng tính đến ngày 01/10/2021; Đồng thời đạt các điều kiện ngưỡng điểm cho từng loại chứng chỉ như sau: SAT (≥ 1440/2400 hoặc ≥ 960/1600), A-Level (điểm mỗi môn thi theo 3 môn trong tổ hợp ≥ C(E-A*)), IB (≥ 24/42), ACT (≥ 21/36)\n\nCó chứng chỉ tiếng Anh IELTS ≥ 5.0 (hoặc chứng chỉ quốc tế khác tương đương) còn thời hạn trong vòng 2 năm tính đến ngày 01/10/2021. Thí sinh không có chứng chỉ tiếng Anh quốc tế tương đương IELTS 5.0 trở lên còn thời hạn theo quy định của TDTU phải đăng ký dự thi Năng lực tiếng Anh do T

In [ ]:
for doc in docs:
  print(doc)
  print('----------------------')
# print(docs2)

page_content='Mã ngành:

7380107

Văn bằng:

Cử nhân ngành Luật Kinh tế

Tổ hợp môn:

A00: Toán – Lý – Hóa

A01: Toán – Lý – Tiếng Anh

C00: Ngữ Văn – Lịch Sử – Địa Lý

D01: Toán – Ngữ Văn – Tiếng Anh

Thời gian đào tạo:

4 năm

40. Luật

Ngành Luật tại VLU định hướng đào tạo ứng dụng, kết hợp lý thuyết với các hoạt động mô phỏng và thực hành thực tế như phiên tòa giả định, ngày hội pháp luật và tham gia phiên tòa thực tế.

Mã ngành:

7380101

Văn bằng:

Cử nhân ngành Luật

Tổ hợp môn:

A00: Toán – Lý – Hóa

A01: Toán – Lý – Tiếng Anh

C00: Ngữ Văn – Lịch Sử – Địa Lý

D01: Toán – Văn – Tiếng Anh

Thời gian đào tạo:

4 năm

41. Mạng máy tính và truyền thông dữ liệu

Ngành Mạng máy tính và truyền thông dữ liệu đáp ứng nhu cầu ngày càng lớn của công nghệ thông tin, với tiềm năng phát triển lớn cho những nhân tài trong lĩnh vực này.

Mã ngành:

7480102

Văn bằng:

Cử nhân Mạng máy tính và truyền thông dữ liệu

Tổ hợp môn:

A00: Toán – Lý – Hóa

A01: Toán – Lý – Tiếng Anh

D07: Toán – Hóa –

# NV retrieval V2

In [ ]:
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

class UniversityRetrievalStrategy1(BaseRetrievalStrategy):
    def retrieve(self, query: str, k=5) -> list:
        university = self.classifier.UniversityRouting(query).university_code
        if university == None:
            university = "TDTU"
        print(f'Truy vấn thuộc về trường: {university}')

        # Kết nối với Qdrant
        client = QdrantClient(
            url=userdata.get('qdrant_url2'),
            api_key=userdata.get('qdrant_api2'),
            prefer_grpc=True
        )

        # Chuyển truy vấn thành vector embedding
        query_vector = self.embeddings.embed_query(query)

        # Truy vấn Qdrant để lấy điểm và khoảng cách
        results = client.search(
            collection_name=str(university),
            query_vector=query_vector,
            limit=k,
            with_payload=True,  # Lấy thêm payload của các tài liệu
            with_vectors=False  # Không cần vector của tài liệu trong kết quả
        )

        # Trả về danh sách tài liệu kèm theo khoảng cách/score
        docs_with_scores = [
            {
                "id": result.id,
                "document": result.payload,  # Nội dung tài liệu
                "distance": result.score     # Khoảng cách hoặc điểm số
            }
            for result in results
        ]

        return docs_with_scores

### Test query

In [ ]:
# Thực hiện truy vấn thực tế
retriever = UniversityRetrievalStrategy1()
query = "Điểm chuẩn ngành Công Nghệ Thông Tin 2021 TDTU"
docs = retriever.retrieve(query, k=5)
docs

Truy vấn thuộc về trường: TDTU


<ipython-input-146-89eabeccaaef>:23: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


[{'id': '71a57b15-4a32-4d23-baf1-6395bb3a43c9',
  'document': {'metadata': {'source': 'Admission\\TDTU\\2021_tuyển sinh.md'},
   'page_content': 'Đối với chương trình tiêu chuẩn, chất lượng cao và chương trình học 2 năm tại cơ sở: Điểm xét tuyển là tổng điểm trung bình 05 học kỳ của các môn trong tổ hợp xét tuyển. Đối với chương trình đại học bằng tiếng Anh, xét tuyển 05 học kỳ theo điểm trung bình học kỳ. Thời gian đăng ký xét tuyển trực tuyến: dự kiến từ 05/04 – 25/05/2021\n\nThời gian nộp hồ sơ xét tuyển: từ 15/4 - 29/5/2021\n\nChi tiết về quy định xét tuyển theo kết quả học tập 05 Học kỳ đợt 1 (xem chi tiết)\n\nDanh mục ngành tuyển sinh, tổ hợp xét tuyển và ngưỡng điểm nộp hồ sơ xét theo kết quả học tập 05 học kỳ dành cho trường THPT đã ký kết hợp tác với TDTU năm 2021 (xem chi tiết)\n\nDanh mục các Trường THPT ký kết năm 2021 (xem tại đây). Danh mục trường chuyên/năng khiếu (xem tại đây), trường trọng điểm (xem tại đây)\n\nHướng dẫn đăng ký xét tuyển 05 Học kỳ đợt 1 (xem tại đây)\

In [ ]:
for doc in docs:
  print(doc["id"])

16165d25-0f32-4fe2-9e73-458ee487ed58
070798cb-83d2-4c15-8cb6-b78d79ae8898
0d8dd726-ecec-4866-b4d6-1d9bea6b6df1
b1fe818e-869c-4f84-904e-016d4069b117
9864ba05-41ef-4647-867d-32c30efd45f9


In [ ]:
# Thực hiện truy vấn thực tế
retriever = UniversityRetrievalStrategy1()
query = "Điểm chuẩn ngành Công Nghệ Thông Tin 2021 BKU"
docs = retriever.retrieve(query, k=3)

for doc in docs:
  print(doc["id"])
  # print(doc["page_content"])


Truy vấn thuộc về trường: BKU


<ipython-input-31-2a89d18252f5>:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


16165d25-0f32-4fe2-9e73-458ee487ed58
070798cb-83d2-4c15-8cb6-b78d79ae8898
0d8dd726-ecec-4866-b4d6-1d9bea6b6df1


In [ ]:
for doc in docs:
  print(doc["id"])
  print(doc["document"])

16165d25-0f32-4fe2-9e73-458ee487ed58
{'metadata': {'source': '/content/drive/MyDrive/Data/Admission/BKU/2024_Ngành và chỉ tiêu.md'}, 'page_content': 'Điểm chuẩn Đại học Bách Khoa - ĐHQG HCM năm 2024 dựa theo phương thức xét tuyển kết hợp giữa điểm thi ĐGNL, điểm thi tốt nghiệp THPT, điểm học lực THPT đã được công bố đến tất cả thí sinh, chi tiết được đăng tải bên dưới.\n\nCông thức tính điểm xét tuyển kết hợp:\n\nĐiểm XT = (0.7 x Điểm ĐGNL) + (0.2 x Điểm thi TN THPT x 3) + (0.1 x Điểm HL THPT)\n\nĐiểm chuẩn theo phương thức Điểm thi THPT năm 2024'}
070798cb-83d2-4c15-8cb6-b78d79ae8898
{'metadata': {'source': '/content/drive/MyDrive/Data/Admission/BKU/2021_tuyển sinh.md'}, 'page_content': 'Hội đồng tuyển sinh đại học chính quy Trường ĐH Bách khoa (ĐH Quốc gia TP.HCM) đã công bố phương hướng tuyển sinh với 6 phương thức tuyển sinh dự kiến năm 2021.\n\nXét tuyển thẳng theo quy chế của Bộ GD-ĐT: 1-5% tổng chỉ tiêu.\n\nƯu tiên xét tuyển theo quy định của ĐH Quốc gia TP.HCM: 15-25% tổn

In [ ]:
docs

[{'id': '16165d25-0f32-4fe2-9e73-458ee487ed58',
  'document': {'metadata': {'source': '/content/drive/MyDrive/Data/Admission/BKU/2024_Ngành và chỉ tiêu.md'},
   'page_content': 'Điểm chuẩn Đại học Bách Khoa - ĐHQG HCM năm 2024 dựa theo phương thức xét tuyển kết hợp giữa điểm thi ĐGNL, điểm thi tốt nghiệp THPT, điểm học lực THPT đã được công bố đến tất cả thí sinh, chi tiết được đăng tải bên dưới.\n\nCông thức tính điểm xét tuyển kết hợp:\n\nĐiểm XT = (0.7 x Điểm ĐGNL) + (0.2 x Điểm thi TN THPT x 3) + (0.1 x Điểm HL THPT)\n\nĐiểm chuẩn theo phương thức Điểm thi THPT năm 2024'},
  'distance': 0.5101755261421204},
 {'id': '070798cb-83d2-4c15-8cb6-b78d79ae8898',
  'document': {'metadata': {'source': '/content/drive/MyDrive/Data/Admission/BKU/2021_tuyển sinh.md'},
   'page_content': 'Hội đồng tuyển sinh đại học chính quy Trường ĐH Bách khoa (ĐH Quốc gia TP.HCM) đã công bố phương hướng tuyển sinh với 6 phương thức tuyển sinh dự kiến năm 2021.\n\nXét tuyển thẳng theo quy chế của Bộ GD-Đ

In [ ]:
docs[0]["distance"]

0.5101755261421204

In [ ]:
for point in docs:
    print(point)
    print('----------------------')
    print(point["distance"])
    print(point["id"])

{'id': '16165d25-0f32-4fe2-9e73-458ee487ed58', 'document': {'metadata': {'source': '/content/drive/MyDrive/Data/Admission/BKU/2024_Ngành và chỉ tiêu.md'}, 'page_content': 'Điểm chuẩn Đại học Bách Khoa - ĐHQG HCM năm 2024 dựa theo phương thức xét tuyển kết hợp giữa điểm thi ĐGNL, điểm thi tốt nghiệp THPT, điểm học lực THPT đã được công bố đến tất cả thí sinh, chi tiết được đăng tải bên dưới.\n\nCông thức tính điểm xét tuyển kết hợp:\n\nĐiểm XT = (0.7 x Điểm ĐGNL) + (0.2 x Điểm thi TN THPT x 3) + (0.1 x Điểm HL THPT)\n\nĐiểm chuẩn theo phương thức Điểm thi THPT năm 2024'}, 'distance': 0.5101755261421204}
----------------------
0.5101755261421204
16165d25-0f32-4fe2-9e73-458ee487ed58
{'id': '070798cb-83d2-4c15-8cb6-b78d79ae8898', 'document': {'metadata': {'source': '/content/drive/MyDrive/Data/Admission/BKU/2021_tuyển sinh.md'}, 'page_content': 'Hội đồng tuyển sinh đại học chính quy Trường ĐH Bách khoa (ĐH Quốc gia TP.HCM) đã công bố phương hướng tuyển sinh với 6 phương thức tuyển si

In [ ]:
# Thực hiện truy vấn thực tế
retriever = UniversityRetrievalStrategy1()
query = "HCMUTE xét tuyển thẳng, ưu tiên xét tuyển thẳng năm 2024 theo quy định nào?"
docs = retriever.retrieve(query, k=3)

print(docs)

Truy vấn thuộc về trường: HCMUTE


<ipython-input-9-cf49aedfc339>:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


[{'id': '4f0f486a-1aa6-4bbd-85db-9fee4294bbe4', 'document': {'metadata': {'source': '/content/drive/MyDrive/Data/Admission/HCMUTE/Tuyển sinh HCMUTE 2024.txt'}, 'page_content': 'Đề án tuyển sinh Trường Đại học Sư phạm Kỹ thuật TP.HCM (HCMUTE) năm 2024\n\nTrường Đại học Sư phạm Kỹ thuật Thành phố Hồ Chí Minh (HCMUTE) tuyển sinh qua 5 phương thức\n\n1) Phương thức 1: Tuyển thẳng học sinh tốt nghiệp THPT năm 2024\n\n2) Phương thức 2: Ưu tiên xét tuyển học sinh tốt nghiệp THPT năm 2024\n\n3) Phương thức 3: Xét tuyển học bạ THPT đối với thí sinh tốt nghiệp THPT trong 3 năm 2022, 2023 và 2024\n\n4) Phương thức 4: Xét điểm thi Đánh giá năng lực của ĐHQG TP.HCM.\n\n5) Phương thức 5: Xét điểm thi Đánh giá năng lực của ĐHQG TP.HCM.\n\nPhương thức xét tuyển năm 2024 1.\tĐiểm thi THPT Xét bằng Điểm thi tốt nghiệp THPT 2024 Phương thức Xét tuyển dựa vào kết quả kỳ thi Trung học phổ thông năm 2024 vẫn tiến hành theo quy định của Bộ Giáo dục và Đào tạo, thí sinh đăng ký xét tuyển vào Trường Đại học 

# Ground truth
- 1. Test 1 excel

In [ ]:
# Replace with your actual file path
# file_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_tdtu.xlsx"
file_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_all_universities.xlsx"


df = pd.read_excel(file_path)
df


,id_question,question,id_corpus,filename
0,c3e174c5-fd8c-4a62-bc97-a42e14d16c65,Điều kiện tiếng Anh đầu vào chương trình đại h...,014a418b-fded-4bd1-a1b1-931188a656ef,{'source': '/content/drive/MyDrive/Data/Admiss...
1,37a1e510-e546-42a3-b79c-d3417e2e9506,"Điểm chuẩn chứng chỉ SAT, A-Level, IB, ACT của...",027aec04-add2-4a22-80ef-09c0180f1a63,{'source': '/content/drive/MyDrive/Data/Admiss...
2,6e323b5f-dd00-4b04-bee6-3e9025842283,Điểm chuẩn xét tuyển đối tượng 4 vào TDTU năm ...,051a6912-8c83-4f9c-ae8a-bb768448fabc,{'source': '/content/drive/MyDrive/Data/Admiss...
3,af0ad4e5-b9d7-4f48-8454-02256fa82dd8,Điểm cộng học sinh giỏi cấp quốc gia năm 2021 ...,0d5fd951-1887-4954-acd6-de041ed21acf,{'source': '/content/drive/MyDrive/Data/Admiss...
4,bc1731b1-4c87-40f4-8336-cf43f60b11a3,Điều kiện tiếng Anh đầu vào chương trình đại h...,11ba40c4-8834-4caf-b5c5-f3345066b345,{'source': '/content/drive/MyDrive/Data/Admiss...
...,...,...,...,...
753,f30a434d-0521-4917-8a67-aaf67d3bf57f,Điểm chuẩn các ngành Công nghệ thông tin của Đ...,37eb7025-ab34-4cee-ad74-007b550502cc,{'source': '/content/drive/MyDrive/Data/Admiss...
754,114fab48-0180-4ca6-bcf3-28e3f192fd1f,"Điểm chuẩn của FPT tại Đà Nẵng, Cần Thơ, Quy N...",5c7b3b45-c4e7-4033-91f5-9edb38075a7b,{'source': '/content/drive/MyDrive/Data/Admiss...
755,4d1ea739-1a2a-4f94-a18e-f79e09dfb681,Điểm nổi bật của chương trình đào tạo tại Đại ...,7d990e65-2ec3-4a2c-b39f-66c6378d5850,{'source': '/content/drive/MyDrive/Data/Admiss...
756,32da8791-b5a0-4efb-bbb0-f21fc7ea71b7,Tỷ lệ sinh viên FPT có việc làm sau tốt nghiệp...,897c8413-96e2-423a-9c3f-69078d6b1621,{'source': '/content/drive/MyDrive/Data/Admiss...


In [ ]:
df["score"] = 1

## View data

In [ ]:

df.iloc[158]

,158
id_question,8e3d6b0d-61f1-49e8-9c5e-6ec1b606591e
question,"HCMUTE xét tuyển thẳng, ưu tiên xét tuyển thẳn..."
id_corpus,997cb8d7-914d-43a1-8351-7ea990526b9b
filename,{'source': '/content/drive/MyDrive/Data/Admiss...


In [ ]:
df

,id_question,question,id_corpus,filename,score
0,c3e174c5-fd8c-4a62-bc97-a42e14d16c65,Điều kiện tiếng Anh đầu vào chương trình đại h...,014a418b-fded-4bd1-a1b1-931188a656ef,{'source': '/content/drive/MyDrive/Data/Admiss...,1
1,37a1e510-e546-42a3-b79c-d3417e2e9506,"Điểm chuẩn chứng chỉ SAT, A-Level, IB, ACT của...",027aec04-add2-4a22-80ef-09c0180f1a63,{'source': '/content/drive/MyDrive/Data/Admiss...,1
2,6e323b5f-dd00-4b04-bee6-3e9025842283,Điểm chuẩn xét tuyển đối tượng 4 vào TDTU năm ...,051a6912-8c83-4f9c-ae8a-bb768448fabc,{'source': '/content/drive/MyDrive/Data/Admiss...,1
3,af0ad4e5-b9d7-4f48-8454-02256fa82dd8,Điểm cộng học sinh giỏi cấp quốc gia năm 2021 ...,0d5fd951-1887-4954-acd6-de041ed21acf,{'source': '/content/drive/MyDrive/Data/Admiss...,1
4,bc1731b1-4c87-40f4-8336-cf43f60b11a3,Điều kiện tiếng Anh đầu vào chương trình đại h...,11ba40c4-8834-4caf-b5c5-f3345066b345,{'source': '/content/drive/MyDrive/Data/Admiss...,1
...,...,...,...,...,...
755,f30a434d-0521-4917-8a67-aaf67d3bf57f,Điểm chuẩn các ngành Công nghệ thông tin của Đ...,37eb7025-ab34-4cee-ad74-007b550502cc,{'source': '/content/drive/MyDrive/Data/Admiss...,1
756,114fab48-0180-4ca6-bcf3-28e3f192fd1f,"Điểm chuẩn của FPT tại Đà Nẵng, Cần Thơ, Quy N...",5c7b3b45-c4e7-4033-91f5-9edb38075a7b,{'source': '/content/drive/MyDrive/Data/Admiss...,1
757,4d1ea739-1a2a-4f94-a18e-f79e09dfb681,Điểm nổi bật của chương trình đào tạo tại Đại ...,7d990e65-2ec3-4a2c-b39f-66c6378d5850,{'source': '/content/drive/MyDrive/Data/Admiss...,1
758,32da8791-b5a0-4efb-bbb0-f21fc7ea71b7,Tỷ lệ sinh viên FPT có việc làm sau tốt nghiệp...,897c8413-96e2-423a-9c3f-69078d6b1621,{'source': '/content/drive/MyDrive/Data/Admiss...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_question  760 non-null    object
 1   question     760 non-null    object
 2   id_corpus    760 non-null    object
 3   filename     760 non-null    object
 4   score        760 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 29.8+ KB


In [ ]:
df["id_question"]

'c3e174c5-fd8c-4a62-bc97-a42e14d16c65'

In [ ]:
for i in range(len(df)):
    print(df.iloc[i])
    query_id = str(df.iloc[i]["id_question"])
    doc_id = str(df.iloc[i]["id_corpus"])
    print(query_id)
    break

id_question                 c3e174c5-fd8c-4a62-bc97-a42e14d16c65
question       Điều kiện tiếng Anh đầu vào chương trình đại h...
id_corpus                   014a418b-fded-4bd1-a1b1-931188a656ef
filename       {'source': '/content/drive/MyDrive/Data/Admiss...
score                                                          1
Name: 0, dtype: object
c3e174c5-fd8c-4a62-bc97-a42e14d16c65


# NV rag

- Nhớ rerun df to update data.
- 0 -150 - 530 - 590 - 730 - 750 - 759
- Nội pkl 150 với 530 nha. (Giá trị len 150 + 380)


In [ ]:
qrels_dict = defaultdict(dict)
for i in range(len(df)):
    query_id = str(df.iloc[i]["id_question"])
    doc_id = str(df.iloc[i]["id_corpus"])
    qrels_dict[query_id][doc_id] = df.iloc[i]["score"]

qrels = Qrels(qrels_dict, name="scifact")
qrels

DictType[unicode_type,DictType[[unichr x 36],int64]<iv=None>]<iv=None>({0002c317-fd1c-4a36-a034-d4bdeecd08e5: {5b6fb3aa-617d-4f36-a9a8-4a62b76cd5e8: 1}, 011f19cf-1d2b-4e2d-94eb-077a12dc5105: {8839d6a6-6542-4698-a9a9-b34e4330ba6e: 1}, 012a64f6-636d-4fa3-92c6-a9afae637ec8: {17f64d51-61c2-4bd1-85b5-45fbd856cd81: 1}, 01c92f8a-85bd-4444-b9ba-5471fb44fedf: {e8b4fef9-57f8-4ccd-9c0d-08f5d13c1818: 1}, 02080427-a67e-4602-a4fd-ec34b870d7c6: {8806602a-67b7-4ff8-9cf7-b99260e140e6: 1}, 02dc9bbc-91d2-4c30-af08-d2fa997e599a: {7ae31a2a-d310-43a1-9004-cbad29365570: 1}, 02e40846-ce27-4bbb-89cb-d98c68831043: {6c662fdb-6f63-423b-9912-2068a7f2dffb: 1}, 02fe0b90-62d9-4c3a-8225-7930e3fb0d6e: {f48bbeae-943f-4e3d-8350-3aa719178281: 1}, 040d17a3-c72e-4d2e-9e04-aac8643f8a5a: {26ed1aa0-4761-4c27-a389-4bccf35b4cf1: 1}, 04474b4a-773d-4925-992b-b19aad96cbe8: {df012d72-01ee-4b23-9c4a-bd6a25887ac1: 1}, 04bf9c0e-be13-4389-a8a3-c05c98421d8a: {cb495e75-fae5-45d7-a3d7-665729c4c0da: 1}, 04de2f59-47c0-449e-9004-9a615c4f7b6c:

In [ ]:
len(qrels)

758

## Run Nv RAG

In [ ]:
# Thực hiện truy vấn thực tế
retriever = UniversityRetrievalStrategy1()

In [ ]:
import time
import pickle
import os

# save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag" # k = 3
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_k5"

# Đảm bảo thư mục tồn tại
os.makedirs(save_path, exist_ok=True)

run_dict_nv = {}
counter = 0  # Bộ đếm số lần request

for i in range(counter, len(df)):
# for i in range(11):  # Giả sử chạy thử với 3 vòng lặp
    print(i)
    query_id = str(df.iloc[i]["id_question"])
    query = df.iloc[i]["question"]

    results = retriever.retrieve(query, k=5)

    run_dict_nv[query_id] = {
        str(point["id"]): point["distance"]
        for point in results
    }

    counter += 1  # Tăng bộ đếm sau mỗi request
    time.sleep(3)  # Mỗi request thì sleep 3 giây

    if counter % 10 == 0 or counter == (len(df)):  # Cứ 10 lần thì lưu lại or end
        file_path = os.path.join(save_path, f"run_dict_nv_{counter}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(run_dict_nv, f)
        print(f"Đã lưu run_dict_nv vào: {file_path}")

    if counter % 5 == 0:  # Sau mỗi 10 request, sleep 10 giây
        print("Đợi 10 giây để tránh lỗi 429...")
        time.sleep(10)

    print("------------")


0
Truy vấn thuộc về trường: TDTU


<ipython-input-54-2a89d18252f5>:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


------------
1
Truy vấn thuộc về trường: TDTU


<ipython-input-54-2a89d18252f5>:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


------------
2
Truy vấn thuộc về trường: TDTU


...

------------
756
Truy vấn thuộc về trường: FPT


<ipython-input-54-2a89d18252f5>:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


------------
757
Truy vấn thuộc về trường: FPT


<ipython-input-54-2a89d18252f5>:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


Đã lưu run_dict_nv vào: /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_k5/run_dict_nv_758.pkl
------------


## Concate checkpoint

0 -150 - 530 - 590 - 730 - 750 - 759

In [ ]:
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag"
file_names = ["run_dict_nv_150.pkl", "run_dict_nv_530.pkl", "run_dict_nv_590.pkl", "run_dict_nv_730.pkl", "run_dict_nv_750.pkl", "run_dict_nv_759.pkl"]
# Add more file names to this list as needed

run_dict_nv_all = {}
total_len = 0

for file_name in file_names:
    file_path = os.path.join(save_path, file_name)
    try:
        with open(file_path, 'rb') as file:
            run_dict_nv = pickle.load(file)
            run_dict_nv_all.update(run_dict_nv)
            current_len = len(run_dict_nv)
            total_len += current_len
            print(f"Length of {file_name}: {current_len}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

print(f"\nTotal Length: {total_len}")

# Save the combined dictionary
output_file_path = os.path.join(save_path, "combined_run_dict_nv.pkl")
with open(output_file_path, "wb") as f:
    pickle.dump(run_dict_nv_all, f)
print(f"Combined dictionary saved to: {output_file_path}")


Length of run_dict_nv_150.pkl: 150
Length of run_dict_nv_530.pkl: 380
Length of run_dict_nv_590.pkl: 60
Length of run_dict_nv_730.pkl: 140
Length of run_dict_nv_750.pkl: 20
Length of run_dict_nv_759.pkl: 9

Total Length: 759
Combined dictionary saved to: /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag/combined_run_dict_nv.pkl


In [ ]:
# prompt: read file pkl  run_dict_nv_150 from save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag"
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag"
file_path = os.path.join(save_path, "combined_run_dict_nv.pkl")

with open(file_path, 'rb') as file:
  run_dict_nv_759 = pickle.load(file)

len(run_dict_nv_759)


759

In [ ]:
# prompt: xóa key 618a7197-5f3b-4ef6-9d5a-b76ce9efcd46 trong run_dict_nv_759

if "618a7197-5f3b-4ef6-9d5a-b76ce9efcd46" in run_dict_nv_759:
  del run_dict_nv_759["618a7197-5f3b-4ef6-9d5a-b76ce9efcd46"]
  print("Key removed successfully.")
else:
  print("Key not found in the dictionary.")


Key removed successfully.


In [ ]:
len(run_dict_nv_759)

758

In [ ]:
# prompt: print each element dict run_dict_nv_759

for key, value in run_dict_nv_759.items():
    print(f"Key: {key}")
    print(f"Value: {value}")
    print("-" * 20)


Key: c3e174c5-fd8c-4a62-bc97-a42e14d16c65
Value: {'ab44531b-f5a1-4d49-99f2-c5e307de8ca7': 0.7584757804870605, '4c66c44f-76eb-44ff-8fe2-459cfd3c26fd': 0.7508246898651123, '014a418b-fded-4bd1-a1b1-931188a656ef': 0.747164785861969}
--------------------
Key: 37a1e510-e546-42a3-b79c-d3417e2e9506
Value: {'15aa4d9c-40af-441c-a031-f3e7f59e6894': 0.6423951387405396, '997bb553-a5e8-490f-86d3-0e09f9122d22': 0.6350304484367371, 'fee8d42e-5332-4a20-8895-22a933dbcca9': 0.618301510810852}
--------------------
Key: 6e323b5f-dd00-4b04-bee6-3e9025842283
Value: {'7be76cd8-0d5d-4745-ba82-eb3ad07e94d6': 0.6945366263389587, 'cabac667-802b-4cb3-aa0a-66eef66e73e3': 0.6762140989303589, '997bb553-a5e8-490f-86d3-0e09f9122d22': 0.6637195944786072}
--------------------
Key: af0ad4e5-b9d7-4f48-8454-02256fa82dd8
Value: {'fee8d42e-5332-4a20-8895-22a933dbcca9': 0.6219178438186646, '6b2ac4cd-abe9-4c63-8ed7-29a216e496a1': 0.612088143825531, '9947adf0-0e32-495e-a8c0-4e38c0e62dff': 0.6104598641395569}
--------------------

In [ ]:
type(run_dict_nv_759)

dict

In [ ]:
dense_run = Run(run_dict_nv_759, name="Navie RAG")

## View result

In [ ]:
from ranx import evaluate

evaluate(qrels, dense_run, metrics=["precision@3", "mrr@3"], make_comparable=True)

{'precision@3': 0.1055408970976253, 'mrr@3': 0.22889182058047494}

In [ ]:
from ranx import compare

compare(
    qrels=qrels,
    runs=[
        dense_run
    ],
    metrics=["precision@3", "recall@3", "mrr@3", "dcg@3", "ndcg@3"],
)


#    Model        P@3    Recall@3    MRR@3    DCG@3    NDCG@3
---  ---------  -----  ----------  -------  -------  --------
a    Navie RAG  0.106       0.317    0.229    0.251     0.251

### k = 5

In [ ]:
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_k5"
file_path = os.path.join(save_path, "run_dict_nv_758.pkl")

with open(file_path, 'rb') as file:
  run_dict_nv_k5 = pickle.load(file)

len(run_dict_nv_k5)


758

In [ ]:
dense_run = Run(run_dict_nv_k5, name="Navie RAG_k5")

In [ ]:
from ranx import evaluate

evaluate(qrels, dense_run, metrics=["precision@5", "mrr@5"], make_comparable=True)

/usr/local/lib/python3.11/dist-packages/ranx/metrics/precision.py:24: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _precision(qrels[i], run[i], k, rel_lvl)


{'precision@5': 0.08469656992084433, 'mrr@5': 0.2543975373790677}

In [ ]:
from ranx import compare

compare(
    qrels=qrels,
    runs=[
        dense_run
    ],
    metrics=["precision@5", "recall@5", "mrr@5", "dcg@5", "ndcg@5"],
)


#    Model           P@5    Recall@5    MRR@5    DCG@5    NDCG@5
---  ------------  -----  ----------  -------  -------  --------
a    Navie RAG_k5  0.085       0.423    0.254    0.296     0.296

# Hybrid retrieval

## Code

In [ ]:
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings
import google.generativeai as genai
from langchain_qdrant import QdrantVectorStore
import os
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from pathlib import Path
from dotenv import load_dotenv
from typing import List, Dict, Tuple
from langchain_core.documents import Document
from sentence_transformers import CrossEncoder

In [ ]:
from typing import List, Tuple  # Thêm import này để tránh lỗi NameError
import numpy as np
class DocumentReRanker:
    def __init__(self, model_name="cross-encoder/ms-marco-MiniLM-L-12-v2"):
        self.cross_encoder = CrossEncoder(model_name)

    def re_rank(self, query: str, documents: List[Document], top_k: int = 5) -> List[Tuple[Document, float]]:
        """
        Re-rank documents based on their relevance to the query.

        :param query: Search query string
        :param documents: List of documents to re-rank
        :param top_k: Number of top documents to return
        :return: List of tuples (document, score)
        """
        pairs = [[query, doc.page_content] for doc in documents]
        scores = self.cross_encoder.predict(pairs)

        min_score = np.min(scores)
        max_score = np.max(scores)
        normalized_scores = [(score - min_score) / (max_score - min_score) for score in scores]

        # Sắp xếp theo điểm score giảm dần
        scored_docs = list(zip(documents, normalized_scores))
        scored_docs.sort(key=lambda x: x[1], reverse=True)

        return scored_docs[:top_k]  # Trả về danh sách (Document, Score)


In [ ]:
class BaseRetrievalStrategy:
    def __init__(self, llm=None):
        self.embeddings = embedding_model
        self.llm = llm or ChatVertexAI(
                                model="gemini-1.5-pro",
                                streaming= True,
                                temperature=0.2,
                                credentials=credentials,
                                max_retries=5
                            )
        self.classifier = QueryRouter()
        self.re_ranker = DocumentReRanker()

    def retrieve(self, query, k=5):
        return self.db.similarity_search(query, k=k)

class UniversityRetrievalStrategy_HB(BaseRetrievalStrategy):
    def __init__(self, llm=None):
        super().__init__(llm)
        self.bm25_docs: Dict[str, List[Document]] = {}  # Cache for BM25 documents per university

    def _init_retrievers(self, university: str, documents: List[Document]):
        # Initialize BM25 retriever if not already cached
        if university not in self.bm25_docs:
            self.bm25_docs[university] = documents
            self.bm25_retriever = BM25Retriever.from_documents(documents)

        # Initialize vector store retriever
        client = QdrantClient(
               url=userdata.get('qdrant_url2'),
               api_key=userdata.get('qdrant_api2'),
               prefer_grpc=True
          )

        vector_store = QdrantVectorStore(
            client=client,
            collection_name=str(university),
            embedding=embedding_model
        )
        vector_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

        # Create ensemble retriever
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, vector_retriever],
            weights=[0.5, 0.5]  # Equal weights for both retrievers
        )

    def retrieve(self, query: str, k: int = 5) -> List[Tuple[Document, float]]:
        try:
          university = self.classifier.UniversityRouting(query).university_code
        except:
          university = "TDTU"

        print(f'Truy vấn thuộc về trường: {university}')

        documents = self._get_university_documents(university)
        self._init_retrievers(university, documents)

        initial_results = self.ensemble_retriever.invoke(query)
        re_ranked_results = self.re_ranker.re_rank(query, initial_results, k)

        # In kết quả cùng score

        return re_ranked_results  # Trả về danh sách (Document, Score)

    def _get_university_documents(self, university: str) -> List[Document]:
        """
        Implement this method to get all documents for a specific university.
        This could be from your database, file system, or other storage.
        """
        # Example implementation - replace with your actual document retrieval logic
        client = QdrantClient(
               url=userdata.get('qdrant_url2'),
               api_key=userdata.get('qdrant_api2'),
               prefer_grpc=True
          )

        vector_store = QdrantVectorStore(
            client=client,
            collection_name=str(university),
            embedding=embedding_model
        )

        # Get all documents for the university
        results = vector_store.similarity_search("", k=150)  # Adjust the number as needed
        return results

## Test query

In [ ]:
# query = "Điều kiện tiếng Anh đầu vào chương trình đại học bằng tiếng Anh của VLU năm 2022 là gì?"
query = "Cơ sở vật chất của trường Tôn đức thắng tại TP. Hồ Chí Minh năm 2024 như thế nào?"
retriever = UniversityRetrievalStrategy_HB()
docs_with_scores = retriever.retrieve(query)



Truy vấn thuộc về trường: TDTU


In [ ]:
# prompt: get id: docs_with_scores[0][0]

docs_with_scores[0][0].metadata["_id"]


'34d5e7fe-710f-4fc7-90dc-e4cb15dc541a'

In [ ]:
docs_with_scores

[(Document(metadata={'source': '/content/drive/MyDrive/Data/Admission/FTU2/Giới thiệu.md', '_id': '9823e18b-5de4-4f2d-81fe-f2a8cbdca235', '_collection_name': 'FTU2'}, page_content='Trường Đại học Ngoại Thương Cơ sở II\n\nTiếng Anh: Foreign Trade University Ho Chi Minh City Campus (FTU2) Trường Đại học Ngoại Thương cơ sở II là cơ sở đào tạo phía Nam của Trường Đại học Ngoại thương tại Hà Nội, đại học chuyên ngành kinh tế đầu ngành tại Việt Nam và thành viên của Bộ Giáo dục và Đào tạo. Cơ sở này được thành lập dựa trên nhu cầu đào tạo cán bộ trong lĩnh vực kinh tế và kinh doanh quốc tế tại các tỉnh thành phía Nam trong giai đoạn hội nhập kinh tế quốc tế.\n\nCơ sở được thành lập theo Quyết định số 1485/GD-ĐT ngày 16/07/1993 của Bộ trưởng Bộ Giáo dục và Đào tạo Việt Nam.\n\nTên trường: Trường Đại học Ngoại thương (Cơ sở 2)\n\nTên tiếng Anh: Foreign Trade University (FTU)\n\nMã trường: NTS\n\nTrực thuộc: Bộ Giáo dục và Đào tạo\n\nLoại trường: Công lập\n\nLoại hình đào tạo: Đại học – Sau

In [ ]:
docs_with_scores[0][1]

1.0

In [ ]:
# prompt: for list(point) in docs_with_scores:
#   print(point)
#   print(point.metadata["_id"])

import numpy as np

for point in docs_with_scores:
    print(point[0].metadata["_id"])
    print(point[1])
    print("---")



34d5e7fe-710f-4fc7-90dc-e4cb15dc541a
1.0
---
bc458917-3a32-4073-94c8-4c1a21a473ce
0.87809974
---
2e7fe435-ef47-4bea-890b-57545fa5dc65
0.6791404
---


## Run

- Hybrdid k =3: 50 - 420 - 500 - 750 - 758
- k = 5: 430 - 580
```
# This is formatted as code
```



In [ ]:
retriever_hb = UniversityRetrievalStrategy_HB()

In [ ]:
# save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid"
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_k5"

# Đảm bảo thư mục tồn tại
os.makedirs(save_path, exist_ok=True)

run_dict_hb = {}
counter = 580  # Bộ đếm số lần request

# Bắt đầu đo tổng thời gian
total_start_time = time.time()

for i in range(counter, len(df)):
    query_id = str(df.iloc[i]["id_question"])
    query = df.iloc[i]["question"]

    print(f"Processing index: {i}")
    print(f"{query}")

    results = retriever_hb.retrieve(query)

    run_dict_hb[query_id] = {
        str(point[0].metadata["_id"]): point[1]
        for point in results
    }

    counter += 1  # Tăng bộ đếm sau mỗi request
    time.sleep(3)  # Mỗi request thì sleep 3 giây

    if counter % 10 == 0 or counter == len(df):  # Cứ 10 lần thì lưu lại
        file_path = os.path.join(save_path, f"run_dict_hb_{counter}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(run_dict_hb, f)
        print(f"Đã lưu run_dict_hb vào: {file_path}")

    if counter % 5 == 0:  # Sau mỗi 5 request, sleep 10 giây
        print("Đợi 10 giây để tránh lỗi 429...")
        time.sleep(10)

    print("--------")

# Kết thúc đo tổng thời gian
total_end_time = time.time()
total_elapsed_time = total_end_time - total_start_time
print(f"Tổng thời gian chạy: {total_elapsed_time:.2f} giây")


Processing index: 580
Trường VLU có những tổ hợp xét tuyển nào cho ngành Công nghệ thông tin năm 2022?
Truy vấn thuộc về trường: VLU
--------
Processing index: 581
Điểm chuẩn ngành Thiết kế Công nghiệp (Thiết kế Sản phẩm) của VLU năm 2021, 2022, 2023, 2024 là bao nhiêu?
Truy vấn thuộc về trường: VLU
--------
Processing index: 582
Điểm chuẩn ngành Công nghệ Kỹ thuật Ô tô tại VLU năm 2021, 2022, 2023, 2024 là bao nhiêu?
Truy vấn thuộc về trường: VLU
--------
Processing index: 583
Điểm chuẩn ngành Luật của VLU năm 2021, 2022, 2023, 2024 là bao nhiêu?
Truy vấn thuộc về trường: VLU
--------
Processing index: 584
Điểm chuẩn ngành Du lịch của VLU năm 2023, 2024 là bao nhiêu?
Truy vấn thuộc về trường: VLU
Đợi 10 giây để tránh lỗi 429...
--------
Processing index: 585
Điểm chuẩn ngành Piano của VLU năm 2023, tổ hợp N00 là bao nhiêu?
Truy vấn thuộc về trường: VLU
--------
Processing index: 586
VLU có bao nhiêu cơ sở đào tạo và ký túc xá ở TP. HCM?
Truy vấn thuộc về trường: VLU
--------
Processin

Truy vấn thuộc về trường: UEH
Đợi 10 giây để tránh lỗi 429...
--------
Processing index: 665
UEH có những địa điểm học nào cho năm 2024?
Truy vấn thuộc về trường: UEH
--------
Processing index: 666
UEH chính thức được thành lập theo quyết định của Thủ tướng Phạm Văn Đồng vào năm nào?
Truy vấn thuộc về trường: UEH
--------
Processing index: 667
Điểm chuẩn của UEH vào năm 2021, 2022, 2023, 2024 là bao nhiêu?
Truy vấn thuộc về trường: UEH
--------
Processing index: 668
Liên hệ UEH về chương trình cử nhân quốc tế Mekong năm 2024 như thế nào?
Truy vấn thuộc về trường: UEH
--------
Processing index: 669
Điểm chuẩn của các trường thành viên thuộc Đại học UEH trong năm 2021-2022 là bao nhiêu?
Truy vấn thuộc về trường: UEH
Đã lưu run_dict_hb vào: /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_k5/run_dict_hb_670.pkl
Đợi 10 giây để tránh lỗi 429...
--------
Processing index: 670
Năm 1996, trường nào được hợp nhất để thành lập UEH thuộc Đại học Quốc gia TP.HCM?
Truy vấn thu

## Concate checkpoint

In [ ]:
# prompt: nối các file này lại pkl

import os
import pickle
# 50 - 420 - 500 - 758
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid"
file_names = ["run_dict_hb_50.pkl", "run_dict_hb_420.pkl", "run_dict_hb_500.pkl", "run_dict_hb_758.pkl"]
# Add more file names to this list as needed

run_dict_hb_all = {}
total_len = 0

for file_name in file_names:
    file_path = os.path.join(save_path, file_name)
    try:
        with open(file_path, 'rb') as file:
            run_dict_nv = pickle.load(file)
            run_dict_hb_all.update(run_dict_nv)
            current_len = len(run_dict_nv)
            total_len += current_len
            print(f"Length of {file_name}: {current_len}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

print(f"\nTotal Length: {total_len}")

# Save the combined dictionary
output_file_path = os.path.join(save_path, "combined_run_dict_hb.pkl")
with open(output_file_path, "wb") as f:
    pickle.dump(run_dict_hb_all, f)
print(f"Combined dictionary saved to: {output_file_path}")


Length of run_dict_hb_50.pkl: 50
Length of run_dict_hb_420.pkl: 370
Length of run_dict_hb_500.pkl: 80
Length of run_dict_hb_758.pkl: 258

Total Length: 758
Combined dictionary saved to: /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid/combined_run_dict_hb.pkl


In [ ]:
hybrid_run_HB = Run(run_dict_hb_all, name="hybrid")

In [ ]:
evaluate(qrels, hybrid_run_HB, metrics=["precision@3", "mrr@3"], make_comparable=True)

{'precision@3': 0.10817941952506596, 'mrr@3': 0.24230430958663152}

## k = 5

In [ ]:
import os
import pickle

def combine_pickle_files(save_path, file_names, output_file):
    run_dict_hb_all = {}
    total_len = 0

    for file_name in file_names:
        file_path = os.path.join(save_path, file_name)
        try:
            with open(file_path, 'rb') as file:
                run_dict_nv = pickle.load(file)
                run_dict_hb_all.update(run_dict_nv)
                current_len = len(run_dict_nv)
                total_len += current_len
                print(f"Length of {file_name}: {current_len}")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error loading {file_path}: {e}")

    print(f"\nTotal Length: {total_len}")

    output_file_path = os.path.join(save_path, output_file)
    with open(output_file_path, "wb") as f:
        pickle.dump(run_dict_hb_all, f)
    print(f"Combined dictionary saved to: {output_file_path}")

# Example usage
# 430 - 580


In [ ]:
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_k5"
file_names = ["run_dict_hb_430.pkl", "run_dict_hb_580.pkl", "run_dict_hb_758.pkl"]
output_file = "combined_run_dict_hb.pkl"

combine_pickle_files(save_path, file_names, output_file)

Length of run_dict_hb_430.pkl: 430
Length of run_dict_hb_580.pkl: 150
Length of run_dict_hb_758.pkl: 178

Total Length: 758
Combined dictionary saved to: /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_k5/combined_run_dict_hb.pkl


In [ ]:
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_k5"
file_path = os.path.join(save_path, "combined_run_dict_hb.pkl")

with open(file_path, 'rb') as file:
  run_dict_hb_k5 = pickle.load(file)

len(run_dict_hb_k5)


758

In [ ]:
hybrid_run_hb_k5 = Run(run_dict_hb_k5, name="hybrid_k5")

In [ ]:
evaluate(qrels, hybrid_run_hb_k5, metrics=["precision@5", "mrr@5"], make_comparable=True)

{'precision@5': 0.08100263852242745, 'mrr@5': 0.2619832893579595}

# Final result
- Đây là DB: dùng recursive chunking


## K = 3

In [ ]:
from ranx import compare

compare(
    qrels=qrels,
    runs=[
        dense_run,
        hybrid_run_HB
    ],
    metrics=["precision@3", "recall@3", "mrr@3", "dcg@3", "ndcg@3"],
)


#    Model        P@3    Recall@3    MRR@3    DCG@3    NDCG@3
---  ---------  -----  ----------  -------  -------  --------
a    Navie RAG  0.106       0.317    0.229    0.251     0.251
b    hybrid     0.108       0.325    0.242    0.263     0.263

## k = 5

In [ ]:
from ranx import compare

compare(
    qrels=qrels,
    runs=[
        dense_run,
        hybrid_run_hb_k5
    ],
    metrics=["precision@5", "recall@5", "mrr@5", "dcg@5", "ndcg@5"],
)


#    Model           P@5    Recall@5    MRR@5    DCG@5    NDCG@5
---  ------------  -----  ----------  -------  -------  --------
a    Navie RAG_k5  0.085       0.423    0.254    0.296     0.296
b    hybrid_k5     0.081       0.405    0.262    0.298     0.298

# New DB
- DB: Sematic chunk


## Navie

In [ ]:
def retrieve_and_save_results_NV(retriever, df, save_path, counter=0, k=5, sleep_time=3, save_interval=10, long_sleep_interval=5, long_sleep_time=10):
    os.makedirs(save_path, exist_ok=True)
    run_dict_nv = {}
    counter = counter  # Bộ đếm số lần request
    print(f"len: {len(df)}")
    for i in range(counter, len(df)):

        query_id = str(df.iloc[i]["id_question"])
        query = df.iloc[i]["question"]
        print(i)
        print(query)
        #----
        results = retriever.retrieve(query, k=k)

        run_dict_nv[query_id] = {
            str(point["id"]): point["distance"]
            for point in results
        }

        counter += 1  # Tăng bộ đếm sau mỗi request
        time.sleep(sleep_time)  # Mỗi request thì sleep

        if counter % save_interval == 0 or counter == len(df):  # Cứ n lần thì lưu lại hoặc khi kết thúc
            file_path = os.path.join(save_path, f"run_dict_nv_{counter}.pkl")
            with open(file_path, "wb") as f:
                pickle.dump(run_dict_nv, f)
            print(f"Đã lưu run_dict_nv vào: {file_path}")

        if counter % long_sleep_interval == 0:  # Sau mỗi n request, sleep lâu hơn để tránh lỗi 429
            print(f"Đợi {long_sleep_time} giây để tránh lỗi 429...")
            time.sleep(long_sleep_time)

        print("------------")


In [ ]:
file_path = "/content/drive/MyDrive/Data/Data_GT_SemC/output_all_universities.xlsx"


df = pd.read_excel(file_path)
df


,id_question,question,id_corpus,filename
0,c2fe54eb-81e6-42b7-b9de-a606ead4d969,Điểm chuẩn các ngành của trường HCMUE năm 2021...,059d597d-53fe-4593-bfb9-d901ce9f9af1,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
1,7b83b525-52c8-4e49-8f9c-cf2bc1dbe2fb,Điểm chuẩn các ngành Giáo dục của trường HCMUE...,24c42eb7-4458-4c9b-a4cf-923c36ccfcb2,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
2,34fe1bba-588f-4e45-818f-cc243f1d558a,Điểm chuẩn xét tuyển bằng điểm thi THPT năm 20...,58719923-1848-4ec1-9dc5-6aa24db96a9e,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
3,de2e3170-a555-44ea-b09d-431497685c8d,Chỉ tiêu xét tuyển từ KQ thi THPT vào ngành Sư...,b7f932d2-b916-4d96-b848-1656fc45d0c8,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
4,b27f0e9c-9a3b-4693-bd6c-68f5f0cfadc4,HCMUE xét tuyển dựa trên kết quả thi tốt nghiệ...,d9fbfe54-7fbc-4a7e-ba2a-02d397d648bf,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
...,...,...,...,...
583,d37a9f86-8876-4f30-bae8-1f6f7b6c364b,Điểm chuẩn ngành Khoa học máy tính của TDTU nă...,5d175dc8-bc44-4abf-bdb7-385622cc2798,{'source': 'Admission\\TDTU\\2024_Điểm chuẩn.md'}
584,0c496787-1cf6-493d-8021-759cc3f1bb2f,Điều kiện tiếng Anh đầu vào chương trình đại h...,8e52f7c2-b43f-4af0-8f74-98a0c561a7ee,{'source': 'Admission\\TDTU\\2024_Điểm chuẩn.md'}
585,5f6d5068-aadf-49e1-b59c-86d47dee12ea,TDTU xét tuyển bổ sung đợt 2 năm 2024 bằng nhữ...,e363c208-4b06-4e33-bb16-f55af96468cd,{'source': 'Admission\\TDTU\\2024_Điểm chuẩn.md'}
586,56ed178e-51b5-42db-9b2b-811fc9a77810,Điểm chuẩn ngành Khoa học máy tính của TDTU nă...,c686783c-9ded-43ba-b7c3-04202fa2fccc,{'source': 'Admission\\TDTU\\2023_Điểm chuẩn.md'}


In [ ]:
df["score"] = 1

In [ ]:
df.iloc[559]

,559
id_question,f09de911-b7f4-4ec0-be8e-543f50c41d69
question,Giới thiệu cơ sở vật chất của Trường đại học T...
id_corpus,25b05fc7-5156-4bf8-a578-6e6b2c1c2fa9
filename,{'source': 'Admission\\TDTU\\Giới thiệu.md'}


- 70 - 100 - 110 - 160 - 170 - 250 - 330 - 350 - 420 - 550 - 580 - 587

In [ ]:
# Giả sử bạn đã khởi tạo retriever và df trước đó
retriever = UniversityRetrievalStrategy1()  # Thay thế bằng class phù hợp
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_Sem_k5" # Semantic chunk
counter = 580
retrieve_and_save_results_NV(retriever, df, save_path, counter)

len: 588
580
Điểm chuẩn chứng chỉ SAT, A-Level, IB, ACT để xét tuyển vào TDTU năm 2021 là gì?
Truy vấn thuộc về trường: TDTU


<ipython-input-146-89eabeccaaef>:23: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


------------
581
Điều kiện xét tuyển theo học bạ 5 học kỳ vào Trường đại học tôn đức thắng TDTU năm 2021 là gì?
Truy vấn thuộc về trường: TDTU


<ipython-input-146-89eabeccaaef>:23: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


------------
582
Điều kiện tuyển sinh ngành liên kết quốc tế tại TDTU năm 2024 như thế nào?
Truy vấn thuộc về trường: TDTU


...

------------
586
Điểm chuẩn ngành Khoa học máy tính của TDTU năm 2024 là bao nhiêu?
Truy vấn thuộc về trường: TDTU


<ipython-input-146-89eabeccaaef>:23: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


------------
587
Điểm chuẩn các ngành liên kết quốc tế của TDTU năm 2023 là bao nhiêu?
Truy vấn thuộc về trường: TDTU


<ipython-input-146-89eabeccaaef>:23: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


Đã lưu run_dict_nv vào: /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_Sem_k5/run_dict_nv_588.pkl
------------


In [ ]:
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_k5"
file_names = ["run_dict_hb_430.pkl", "run_dict_hb_580.pkl", "run_dict_hb_758.pkl"]
output_file = "combined_run_dict_hb.pkl"

combine_pickle_files(save_path, file_names, output_file)

In [ ]:
# prompt: write file_names 70 - 100 - 110 - 160 - 170 - 250 - 330 - 350 - 420 - 550 - 580 - 587 link this: run_dict_nv_430

file_names = ["run_dict_nv_70.pkl", "run_dict_nv_100.pkl", "run_dict_nv_110.pkl",
              "run_dict_nv_160.pkl", "run_dict_nv_170.pkl", "run_dict_nv_250.pkl",
              "run_dict_nv_330.pkl", "run_dict_nv_350.pkl", "run_dict_nv_420.pkl",
              "run_dict_nv_550.pkl", "run_dict_nv_580.pkl", "run_dict_nv_588.pkl",]

save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_Sem_k5"
output_file = "combined_run_dict_nv_k5_sem.pkl"

combine_pickle_files(save_path, file_names, output_file)


Length of run_dict_nv_70.pkl: 70
Length of run_dict_nv_100.pkl: 30
Length of run_dict_nv_110.pkl: 10
Length of run_dict_nv_160.pkl: 50
Length of run_dict_nv_170.pkl: 10
Length of run_dict_nv_250.pkl: 80
Length of run_dict_nv_330.pkl: 80
Length of run_dict_nv_350.pkl: 20
Length of run_dict_nv_420.pkl: 70
Length of run_dict_nv_550.pkl: 130
Length of run_dict_nv_580.pkl: 30
Length of run_dict_nv_588.pkl: 8

Total Length: 588
Combined dictionary saved to: /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_Sem_k5/combined_run_dict_nv_k5_sem.pkl


In [ ]:
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/NV_rag_Sem_k5"
file_path = os.path.join(save_path, "combined_run_dict_nv_k5_sem.pkl")

with open(file_path, 'rb') as file:
  run_dict_nv_k5 = pickle.load(file)

len(run_dict_nv_k5)


588

In [ ]:
# prompt: print key and value run_dict_nv_k5 just 5 key

for key, value in list(run_dict_nv_k5.items())[:5]:
    print(f"Key: {key}")
    print(f"Value: {value}")
    print("-" * 20)


Key: c2fe54eb-81e6-42b7-b9de-a606ead4d969
Value: {'3908fde3-a010-4276-af35-105f91cac8db': 0.5339617729187012, '059d597d-53fe-4593-bfb9-d901ce9f9af1': 0.4876854419708252, '8527a511-4423-4112-84eb-132b4fbbe119': 0.464850515127182, 'b5f30df2-7609-4f7b-acc8-924de2d70698': 0.46317946910858154, '1ab332e6-e287-4a3c-b312-4b732bbc1e49': 0.45226597785949707}
--------------------
Key: 7b83b525-52c8-4e49-8f9c-cf2bc1dbe2fb
Value: {'36d350fd-81ec-4607-a4d5-dd5b16620080': 0.6466678380966187, '3908fde3-a010-4276-af35-105f91cac8db': 0.5894885063171387, '2068c501-f681-4b46-858f-d355ed48269d': 0.5758955478668213, 'abe9c926-a54e-4e87-8d3f-444d325d86b9': 0.5691916942596436, 'd820cbd0-3f54-4418-9ae0-11619f4f1332': 0.5554969310760498}
--------------------
Key: 34fe1bba-588f-4e45-818f-cc243f1d558a
Value: {'3908fde3-a010-4276-af35-105f91cac8db': 0.5460814237594604, 'b5f30df2-7609-4f7b-acc8-924de2d70698': 0.474494993686676, '2068c501-f681-4b46-858f-d355ed48269d': 0.46355926990509033, '1ab332e6-e287-4a3c-b312-4b

In [ ]:
# prompt: create run_dict_nv_k3. Get key and value but value is dict and it have 5 element, get 3 element first

run_dict_nv_k3 = {}
for key, value in list(run_dict_nv_k5.items()):
  run_dict_nv_k3[key] = dict(list(value.items())[:3])


In [ ]:
len(run_dict_nv_k3)

588

## Hybrid
- DB new: Semantic chunk

In [ ]:
import os
import pickle
import time

def retrieve_and_save_results_HB(retriever, df, save_path, counter=0, sleep_time=3, save_interval=10, long_sleep_interval=5, long_sleep_time=10 ):
    os.makedirs(save_path, exist_ok=True)
    run_dict = {}
    counter = counter  # Bộ đếm số lần request

    # Bắt đầu đo tổng thời gian
    total_start_time = time.time()

    for i in range(counter, len(df)):
        query_id = str(df.iloc[i]["id_question"])
        query = df.iloc[i]["question"]

        print(f"Processing index: {i}")
        print(f"{query}")

        results = retriever.retrieve(query)

        run_dict[query_id] = {
            str(point[0].metadata["_id"]): point[1]
            for point in results
        }

        counter += 1  # Tăng bộ đếm sau mỗi request
        time.sleep(sleep_time)  # Mỗi request thì sleep

        if counter % save_interval == 0 or counter == len(df):  # Cứ n lần thì lưu lại
            file_path = os.path.join(save_path, f"run_dict_{counter}.pkl")
            with open(file_path, "wb") as f:
                pickle.dump(run_dict, f)
            print(f"Đã lưu run_dict vào: {file_path}")

        if counter % long_sleep_interval == 0:  # Sau mỗi n request, sleep lâu hơn để tránh lỗi 429
            print(f"Đợi {long_sleep_time} giây để tránh lỗi 429...")
            time.sleep(long_sleep_time)

        print("--------")

    # Kết thúc đo tổng thời gian
    total_end_time = time.time()
    total_elapsed_time = total_end_time - total_start_time
    print(f"Tổng thời gian chạy: {total_elapsed_time:.2f} giây")


In [ ]:
file_path = "/content/drive/MyDrive/Data/Data_GT_SemC/output_all_universities.xlsx"


df = pd.read_excel(file_path)
df[:5]

,id_question,question,id_corpus,filename
0,c2fe54eb-81e6-42b7-b9de-a606ead4d969,Điểm chuẩn các ngành của trường HCMUE năm 2021...,059d597d-53fe-4593-bfb9-d901ce9f9af1,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
1,7b83b525-52c8-4e49-8f9c-cf2bc1dbe2fb,Điểm chuẩn các ngành Giáo dục của trường HCMUE...,24c42eb7-4458-4c9b-a4cf-923c36ccfcb2,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
2,34fe1bba-588f-4e45-818f-cc243f1d558a,Điểm chuẩn xét tuyển bằng điểm thi THPT năm 20...,58719923-1848-4ec1-9dc5-6aa24db96a9e,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
3,de2e3170-a555-44ea-b09d-431497685c8d,Chỉ tiêu xét tuyển từ KQ thi THPT vào ngành Sư...,b7f932d2-b916-4d96-b848-1656fc45d0c8,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
4,b27f0e9c-9a3b-4693-bd6c-68f5f0cfadc4,HCMUE xét tuyển dựa trên kết quả thi tốt nghiệ...,d9fbfe54-7fbc-4a7e-ba2a-02d397d648bf,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...


In [ ]:
# Giả sử bạn đã khởi tạo retriever và df trước đó
retriever_hb = UniversityRetrievalStrategy_HB()  # Thay thế bằng class phù hợp
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_Sem_k5" # Semantic chunk
counter = 0
retrieve_and_save_results_HB(retriever_hb, df, save_path, counter)

Processing index: 0
Điểm chuẩn các ngành của trường HCMUE năm 2021 dựa trên điểm thi tốt nghiệp THPT là bao nhiêu?
Truy vấn thuộc về trường: HCMUE
--------
Processing index: 1
Điểm chuẩn các ngành Giáo dục của trường HCMUE năm 2024 như thế nào?
Truy vấn thuộc về trường: HCMUE
--------
Processing index: 2
Điểm chuẩn xét tuyển bằng điểm thi THPT năm 2021 vào trường HCMUE là bao nhiêu?
Truy vấn thuộc về trường: HCMUE
--------
Processing index: 3
Chỉ tiêu xét tuyển từ KQ thi THPT vào ngành Sư phạm Toán của trường HCMUE năm 2024 là bao nhiêu?
Truy vấn thuộc về trường: HCMUE
--------
Processing index: 4
HCMUE xét tuyển dựa trên kết quả thi tốt nghiệp THPT năm 2021 với những tổ hợp môn nào?
Truy vấn thuộc về trường: HCMUE
Đợi 10 giây để tránh lỗi 429...
--------
Processing index: 5
Chỉ tiêu tuyển sinh ngành Sư phạm Toán học của trường HCMUE năm 2022 là bao nhiêu?
Truy vấn thuộc về trường: HCMUE
--------
Processing index: 6
Điểm chuẩn ngành Sư phạm Lịch sử của trường HCMUE năm 2024 là bao nhiê

In [ ]:
# load result checkpoint
save_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/saveCheckpoint/Hybrid_Sem_k5"
file_path = os.path.join(save_path, "run_dict_588.pkl")

with open(file_path, 'rb') as file:
  run_dict_hb_k5 = pickle.load(file)

len(run_dict_hb_k5)


588

In [ ]:
# prompt: create run_dict_nv_k3. Get key and value but value is dict and it have 5 element, get 3 element first

run_dict_hb_k3 = {}
for key, value in list(run_dict_hb_k5.items()):
  run_dict_hb_k3[key] = dict(list(value.items())[:3])


# Evaluate k = 5,3 database: semantic chunk

In [ ]:
file_path = "/content/drive/MyDrive/Data/Data_GT_SemC/output_all_universities.xlsx"


df = pd.read_excel(file_path)
df

,id_question,question,id_corpus,filename
0,c2fe54eb-81e6-42b7-b9de-a606ead4d969,Điểm chuẩn các ngành của trường HCMUE năm 2021...,059d597d-53fe-4593-bfb9-d901ce9f9af1,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
1,7b83b525-52c8-4e49-8f9c-cf2bc1dbe2fb,Điểm chuẩn các ngành Giáo dục của trường HCMUE...,24c42eb7-4458-4c9b-a4cf-923c36ccfcb2,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
2,34fe1bba-588f-4e45-818f-cc243f1d558a,Điểm chuẩn xét tuyển bằng điểm thi THPT năm 20...,58719923-1848-4ec1-9dc5-6aa24db96a9e,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
3,de2e3170-a555-44ea-b09d-431497685c8d,Chỉ tiêu xét tuyển từ KQ thi THPT vào ngành Sư...,b7f932d2-b916-4d96-b848-1656fc45d0c8,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
4,b27f0e9c-9a3b-4693-bd6c-68f5f0cfadc4,HCMUE xét tuyển dựa trên kết quả thi tốt nghiệ...,d9fbfe54-7fbc-4a7e-ba2a-02d397d648bf,{'source': 'Admission\\HCMUE\\DH_SuPham_2021.t...
...,...,...,...,...
583,d37a9f86-8876-4f30-bae8-1f6f7b6c364b,Điểm chuẩn ngành Khoa học máy tính của trường ...,5d175dc8-bc44-4abf-bdb7-385622cc2798,{'source': 'Admission\\TDTU\\2024_Điểm chuẩn.md'}
584,0c496787-1cf6-493d-8021-759cc3f1bb2f,Điều kiện tiếng Anh đầu vào chương trình đại h...,8e52f7c2-b43f-4af0-8f74-98a0c561a7ee,{'source': 'Admission\\TDTU\\2024_Điểm chuẩn.md'}
585,5f6d5068-aadf-49e1-b59c-86d47dee12ea,TDTU xét tuyển bổ sung đợt 2 năm 2024 bằng nhữ...,e363c208-4b06-4e33-bb16-f55af96468cd,{'source': 'Admission\\TDTU\\2024_Điểm chuẩn.md'}
586,56ed178e-51b5-42db-9b2b-811fc9a77810,Điểm chuẩn ngành Khoa học máy tính của TDTU nă...,c686783c-9ded-43ba-b7c3-04202fa2fccc,{'source': 'Admission\\TDTU\\2023_Điểm chuẩn.md'}


In [ ]:
df["score"] = 1

In [ ]:
qrels_dict = defaultdict(dict)
for i in range(len(df)):
    query_id = str(df.iloc[i]["id_question"])
    doc_id = str(df.iloc[i]["id_corpus"])
    qrels_dict[query_id][doc_id] = df.iloc[i]["score"]

qrels = Qrels(qrels_dict, name="scifact")
qrels

DictType[unicode_type,DictType[[unichr x 36],int64]<iv=None>]<iv=None>({00ad21e8-11a6-47ac-83c7-4977346ee58b: {ac66b27b-d9c5-4086-bb86-ad1261a1f4a2: 1}, 00b46e9c-b176-4efe-b04c-0574c3880df7: {9e7b5042-ff84-4082-a3b9-a0f139108b5b: 1}, 00ff33c3-125b-4991-808e-b88ec2c4bbe2: {adeb5dbf-34c3-4fc5-9644-397d0808065b: 1}, 015005c7-0050-4149-a323-8b3ce30bc6c0: {3f34daa4-8747-41c0-ab0b-4d08a278f843: 1}, 01e136f1-1cb9-4125-b69d-8a103ecb80a0: {eaef950f-9443-42e1-90c0-fb9a26ad3ecc: 1}, 027a492f-90ea-4dd5-a3e7-cda879965fc8: {e97390c9-1636-4ee2-be49-f3c4e88dfff6: 1}, 02880769-df56-4fea-8a6b-e2c4995d0e9d: {dff3cbb7-61a8-4d78-8582-ac42c4047cb9: 1}, 02b930e7-6c67-44e2-ace1-249f55e6b7b1: {835b6506-047a-4ddf-8bb2-c21014ee6588: 1}, 0314387e-7d82-491c-83ca-a75cc79c0a32: {dfac48b0-a4ba-430c-b5e9-bc524839d54e: 1}, 036e18cb-5a41-47d3-9b3f-fc98a6c81026: {fc86f349-f9a3-43e8-8260-823171eca675: 1}, 03bc2fed-13eb-4f04-ba4c-80a2496ef0fb: {76da7c95-0afb-4c6d-9ade-c57526719537: 1}, 042407b5-a58d-4796-840e-f833d284e0c8:

In [ ]:
# Navie rag k = 3,5. DB: semantic chunk

In [ ]:
# k = 3
# run_dict_nv_k3, run_dict_nv_k5
# run_dict_hb_k3, run_dict_hb_k5


## K = 3

In [ ]:
navie_rag_k3_sem = Run(run_dict_nv_k3, name="Navie_rag_k3_sem")

In [ ]:
hybrid_rag_k3_sem = Run(run_dict_hb_k3, name="Hybrid_rag_k3_sem")

In [ ]:
from ranx import compare

compare(
    qrels=qrels,
    runs=[
        navie_rag_k3_sem,
        hybrid_rag_k3_sem
    ],
    metrics=["precision@3", "recall@3", "mrr@3", "dcg@3", "ndcg@3"],
)


/usr/local/lib/python3.11/dist-packages/ranx/metrics/precision.py:24: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _precision(qrels[i], run[i], k, rel_lvl)


#    Model                P@3    Recall@3    MRR@3    DCG@3    NDCG@3
---  -----------------  -----  ----------  -------  -------  --------
a    Navie_rag_k3_sem   0.152       0.457    0.337    0.368     0.368
b    Hybrid_rag_k3_sem  0.157       0.471    0.325    0.362     0.362

## k = 5

In [ ]:
navie_rag_k5_sem = Run(run_dict_nv_k5, name="Navie_rag_k5_sem")

In [ ]:
hybrid_rag_k5_sem = Run(run_dict_hb_k5, name="Hybrid_rag_k5_sem")

In [ ]:
from ranx import compare

compare(
    qrels=qrels,
    runs=[
        navie_rag_k5_sem,
        hybrid_rag_k5_sem
    ],
    metrics=["precision@5", "recall@5", "mrr@5", "dcg@5", "ndcg@5"],
)


#    Model                P@5    Recall@5    MRR@5    DCG@5    NDCG@5
---  -----------------  -----  ----------  -------  -------  --------
a    Navie_rag_k5_sem   0.114       0.571    0.363    0.415     0.415
b    Hybrid_rag_k5_sem  0.117       0.587    0.351    0.41      0.41